In [52]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
def preprocess_data(df):
    # Fill NaN values with empty string to avoid issues with TF-IDF
    df = df.fillna("")
    return df

In [55]:
# Load dataset (assuming you have a dataset `wildlife_df` in pandas DataFrame format)
wildlife_df = pd.read_csv("animals.csv")
wildlife_df = preprocess_data(wildlife_df)

In [56]:
# Combine relevant features into a single text column for similarity comparison
wildlife_df["combined_features"] = wildlife_df["Cities Found"] + " " + wildlife_df["Forest Type"] + " " + wildlife_df["Season Active"] + " " + wildlife_df["Day Timing Active"]

# Train TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(wildlife_df["combined_features"].values.astype('U'))

In [57]:
# Save model and vectorizer to pickle file
with open("wildlife_model.pkl", "wb") as model_file:
    pickle.dump((wildlife_df, vectorizer, tfidf_matrix), model_file)

In [58]:
def get_top_animals(location, season, visit_time, top_n=5):
    # Load model from pickle file
    with open("wildlife_model.pkl", "rb") as model_file:
        wildlife_df, vectorizer, tfidf_matrix = pickle.load(model_file)
    
    # User query formatted similarly
    user_query = location + " " + season + " " + visit_time
    user_tfidf = vectorizer.transform([user_query])
    
    # Compute cosine similarity
    similarity_scores = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    
    # Get indices of top N matches
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    
    # Get the top matching animals along with relevant details
    result_df = wildlife_df.iloc[top_indices][["Animal Name", "Forest Type", "Season Active", "Day Timing Active", "Famous Forest for It", "Food", "Preferred Shelter Type", "Unique Fact"]]
    
    # Compute accuracy as the average similarity score of the top matches
    accuracy = similarity_scores[top_indices].mean() * 100  # Convert to percentage
    
    # Convert to HTML table format with custom styling
    table_html = result_df.to_html(index=False, escape=False, classes="styled-table")
    
    # Append accuracy information
    table_html += f"<p><strong>Accuracy of top {top_n} matches: {accuracy:.2f}%</strong></p>"
    
    return table_html


In [59]:
# Prompt user for input
user_location = input("Enter the city/state you are visiting: ")
user_season = input("Enter the season of your visit: ")
user_visit_time = input("Enter the time of day you are visiting (e.g., Crepuscular, Nocturnal, Diurnal): ")

Enter the city/state you are visiting:  shimla
Enter the season of your visit:  summer
Enter the time of day you are visiting (e.g., Crepuscular, Nocturnal, Diurnal):  abc


In [60]:
# Get recommendations
top_animals_html = get_top_animals(user_location, user_season, user_visit_time)

In [50]:
# get the recommendation 
top_animals_html = get_top_animals(df, user_location, user_season, user_visit_time)

In [61]:
# Print HTML with custom styles
html_style = """
<style>
.styled-table {
    width: 100%;
    border-collapse: collapse;
    font-family: Arial, sans-serif;
}
.styled-table th {
    background-color: #4CAF50;
    color: white;
    padding: 10px;
    text-align: left;
    font-size: 16px;
}
.styled-table td {
    padding: 8px;
    border-bottom: 1px solid #ddd;
}
.styled-table tr:nth-child(even) {
    background-color: #f2f2f2;
}
.styled-table tr:nth-child(odd) {
    background-color: #ffffff;
}
</style>
"""
print(html_style + top_animals_html)


<style>
.styled-table {
    width: 100%;
    border-collapse: collapse;
    font-family: Arial, sans-serif;
}
.styled-table th {
    background-color: #4CAF50;
    color: white;
    padding: 10px;
    text-align: left;
    font-size: 16px;
}
.styled-table td {
    padding: 8px;
    border-bottom: 1px solid #ddd;
}
.styled-table tr:nth-child(even) {
    background-color: #f2f2f2;
}
.styled-table tr:nth-child(odd) {
    background-color: #ffffff;
}
</style>
<table border="1" class="dataframe styled-table">
  <thead>
    <tr style="text-align: right;">
      <th>Animal Name</th>
      <th>Forest Type</th>
      <th>Season Active</th>
      <th>Day Timing Active</th>
      <th>Famous Forest for It</th>
      <th>Food</th>
      <th>Preferred Shelter Type</th>
      <th>Unique Fact</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>One-Horned Rhinoceros</td>
      <td>Mangrove</td>
      <td>Summer</td>
      <td>Crepuscular</td>
      <td>Sundarbans</td>
      <td>Herbivore</td>
     